In [ ]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
!pip install transformers datasets wandb peft trl torch accelerate bitsandbytes gradio

In [ ]:
# Import necessary libraries
import gradio as gr
from unsloth import FastLanguageModel
import torch
from transformers import TextStreamer

# Define the model name and other parameters
model_name = "imsanjoykb/deepSQL-R1-distill-8B"
max_seq_length = 2048
dtype = None
load_in_4bit = True

# Load the model and tokenizer from Hugging Face
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# Enable faster inference
FastLanguageModel.for_inference(model)

# Define the prompt template
odoo_text2sql_prompt = """
Instruction: {instruction}
Input: {input_text}
Output: {output_text}
DB Schema: {db_schema}
"""

# Define the database schema
db_schema = """
CREATE TABLE product_product (
    id SERIAL NOT NULL,
    message_main_attachment_id INTEGER,
    product_tmpl_id INTEGER NOT NULL,
    create_uid INTEGER,
    write_uid INTEGER,
    default_code VARCHAR,
    barcode VARCHAR,
    combination_indices VARCHAR,
    volume NUMERIC,
    weight NUMERIC,
    active BOOLEAN,
    can_image_variant_1024_be_zoomed BOOLEAN,
    create_date TIMESTAMP WITHOUT TIME ZONE,
    write_date TIMESTAMP WITHOUT TIME ZONE,
    store_qty_available DOUBLE PRECISION,
    store_standard_price DOUBLE PRECISION,
    store_sales_count DOUBLE PRECISION,
    CONSTRAINT product_product_pkey PRIMARY KEY (id),
    CONSTRAINT product_product_create_uid_fkey FOREIGN KEY(create_uid) REFERENCES res_users (id) ON DELETE SET NULL,
    CONSTRAINT product_product_message_main_attachment_id_fkey FOREIGN KEY(message_main_attachment_id) REFERENCES ir_attachment (id) ON DELETE SET NULL,
    CONSTRAINT product_product_product_tmpl_id_fkey FOREIGN KEY(product_tmpl_id) REFERENCES product_template (id) ON DELETE CASCADE,
    CONSTRAINT product_product_write_uid_fkey FOREIGN KEY(write_uid) REFERENCES res_users (id) ON DELETE SET NULL
)
"""

# Function to generate SQL query
def generate_sql(instruction, input_text):
    output_text = ""
    prompt = odoo_text2sql_prompt.format(
        instruction=instruction,
        input_text=input_text,
        output_text=output_text,
        db_schema=db_schema
    )

    inputs = tokenizer(
        [prompt],
        return_tensors="pt"
    ).to("cuda")

    text_streamer = TextStreamer(tokenizer)
    _ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=350)

    return text_streamer.text

# Function to clear inputs
def clear_inputs():
    return "", ""

# Create the Gradio interface with enhanced features
with gr.Blocks(css="""
    .centered {
        display: flex;
        justify-content: center;
        align-items: center;
        text-align: center;
    }
    .title {
        font-size: 2em;
        font-weight: bold;
        margin-bottom: 20px;
    }
    .description {
        font-size: 1.2em;
        margin-bottom: 20px;
    }
    .button {
        background-color: #007BFF; /* Sea blue color */
        color: white;
        border: none;
        padding: 10px 20px;
        text-align: center;
        text-decoration: none;
        display: inline-block;
        font-size: 16px;
        margin: 4px 2px;
        cursor: pointer;
        border-radius: 12px;
    }
    .button:hover {
        background-color: #0056b3;
    }
""") as demo:
    gr.Markdown('<div class="centered"><div class="title">DeepSQL AI Assistant</div></div>')
    gr.Markdown('<div class="centered"><div class="description">Generate SQL queries for Database Schema based on Natural Language input.</div></div>')

    with gr.Row():
        with gr.Column():
            instruction = gr.Textbox(lines=7, placeholder="Enter the instruction here...", label="Instruction")
            input_text = gr.Textbox(lines=7, placeholder="Enter the input text here...", label="Input Text")
            clear_button = gr.Button("Clear", elem_classes="button")

        with gr.Column():
            output = gr.Textbox(lines=15, placeholder="Generated SQL query will appear here...", label="Output SQL Query")
            feedback = gr.Textbox(lines=2, placeholder="Provide your feedback here...", label="Feedback")

    examples = gr.Examples(
        examples=[
            ["Find the top 5 products with the highest sales count.", "What are the top sales products?"],
            ["List all active products.", "Show me all active products."],
        ],
        inputs=[instruction, input_text]
    )

    submit_button = gr.Button("Generate SQL", elem_classes="button")
    submit_button.click(generate_sql, inputs=[instruction, input_text], outputs=output)
    clear_button.click(clear_inputs, outputs=[instruction, input_text])

# Launch the Gradio interface with sharing enabled
demo.launch(share=True)